#Transform silver into gold layer

In [0]:
query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.customer_id) AS customer_key,
    ci.customer_id,
    ci.customer_key AS crm_customer_key,
    ci.first_name,
    ci.last_name,
    la.country,
    ci.marital_status,
    CASE
        WHEN ci.gender <> 'n/a' THEN ci.gender
        ELSE COALESCE(ca.gender, 'n/a')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.created_date AS create_date
FROM silver.crm_customers ci
LEFT JOIN silver.erpcustomers ca
    ON CAST(ci.customer_id AS STRING) = CAST(ca.customer_number AS STRING)
LEFT JOIN silver.erp_customer_location la
    ON CAST(ci.customer_id AS STRING) = CAST(la.customer_number AS STRING)
"""
df = spark.sql(query)


In [0]:


df.limit(10).display()

customer_key,customer_id,crm_customer_key,first_name,last_name,country,marital_status,gender,birthdate,create_date
1,null,A01Ass,null,null,null,n/a,n/a,null,null
2,null,13451235,null,null,null,n/a,n/a,null,null
3,null,SF566,null,null,null,n/a,n/a,null,null
4,null,PO25,null,null,null,n/a,n/a,null,null
5,11000,AW00011000,Jon,Yang,null,Married,Male,null,2025-10-06
6,11001,AW00011001,Eugene,Huang,null,Single,Male,null,2025-10-06
7,11002,AW00011002,Ruben,Torres,null,Married,Male,null,2025-10-06
8,11003,AW00011003,Christy,Zhu,null,Single,Female,null,2025-10-06
9,11004,AW00011004,Elizabeth,Johnson,null,Single,Female,null,2025-10-06
10,11005,AW00011005,Julio,Ruiz,null,Single,Male,null,2025-10-06


#writing to gold layer

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_customers")